In [1]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1)

In [2]:
contents = ['상처받은 아이들은 너무 일찍 커버려',
           '내가 상처받은 거 아는 사람 불편해',
           '잘 사는 사람들은 좋은 사람 되기 쉬워',
           '아무 일도 아니야 괜찮아']

In [3]:
# Twitter -> Okt
from konlpy.tag import Twitter
t = Twitter()

/Users/jaecheolchoi/opt/anaconda3/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
# 형태소 분석
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

In [5]:
# 다시 문장으로
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

In [6]:
# 벡터라이즈
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [7]:
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [8]:
vectorizer.get_feature_names()

['괜찮아',
 '너무',
 '되기',
 '받은',
 '불편해',
 '사는',
 '사람',
 '상처',
 '쉬워',
 '아는',
 '아니야',
 '아무',
 '아이',
 '일도',
 '일찍',
 '좋은',
 '커버']

In [9]:
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

In [10]:
# 테스트용 문장
new_post = ['상처받기 싫어 괜찮아']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 상처 받기 싫어 괜찮아']

In [11]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()
# 괜찮아, 0, 0, 0, .., '상처', 0, 0, 0, ..

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
# 유클리드 거리
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [13]:
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [14]:
print("Best post is ", dist.index(min(dist)), "dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is  3 dist =  2.0
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


In [15]:
for i in range(0, len(contents)):
    print(X.getrow(i).toarray())

print('-------------------------------------')
print(new_post_vec.toarray())

[[0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1]]
[[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
[[0 0 1 0 0 1 2 0 1 0 0 0 0 0 0 1 0]]
[[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]]
-------------------------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


### TF-IDF

In [16]:
#TfidVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [21]:
# 다시 적용
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [22]:
X.toarray().transpose()

array([[0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.40104275, 0.6191303 , 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.43671931, 0.        , 0.        , 0.        ]])

In [23]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.78528828, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.6191303 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [27]:
# 거리를 구하는 또다른 방법 normailize

def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [28]:
dist = [dist_norm(each, new_post_vec) for each in X]

print("Best post is ", dist.index(min(dist)), "dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is  3 dist =  1.1021396119773588
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


### 실전 크롤링

In [29]:
import json
import datetime
import urllib.request

def gen_search_url(api_node, search_text, start_num, disp_num):
    base = "https://openapi.naver.com/v1/search"
    node = "/" + api_node + ".json"
    param_query = "?query=" + urllib.parse.quote(search_text)
    param_start = "&start=" + str(start_num)
    param_disp = "&display=" + str(disp_num)
    
    return base + node + param_query + param_start + param_disp

def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    
    print("[%s] Url Request Success" % datetime.datetime.now())
    
    return json.loads(response.read().decode("utf-8"))

In [30]:
client_id = "1J8AA25gNpLQl6ddcR8l"
client_secret = "HOYGXlyWjm"

url = gen_search_url('kin', '파이썬', 10, 10)
one_result = get_result_onpage(url)
one_result

[2021-01-08 11:06:39.474926] Url Request Success


{'lastBuildDate': 'Fri, 08 Jan 2021 11:06:39 +0900',
 'total': 163267,
 'start': 10,
 'display': 10,
 'items': [{'title': '<b>파이썬</b> 슬라이싱 관련 질문이요',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=376977428&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=10&search_sort=0&spq=0',
   'description': "string = &quot;홀짝홀짝홀짝&quot;print(string[::2]) <b>파이썬</b>에서 이 코드를 실행시키면 '홀홀홀'이 나오는데 이유가 뭔가요?... 이 의미는 첫 번째 요소(<b>파이썬</b>의 인덱스는 0 부터 시작합니다) 부터 끝까지(-1 은 끝에서 첫 번째를 의미합니다.) 1... "},
  {'title': '아ㅏ어쩌죠 <b>파이썬</b>독학하고 싶은데 먼저 뭐해야하낭ㅇ요',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40608&docId=375044291&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=11&search_sort=0&spq=0',
   'description': '<b>파이썬</b>으로 자격증따고싶어요!! 아직나이ㅣ어리니까 시간도많고 아이구조아라 <b>파이썬</b>시작하려면 뭐부터해야하나요?! <b>파이썬</b>이 조금 익숙해지면 인터넷강의도... C언어 관계없이 <b>파이썬</b> 설치부터, 변수, 자료형 ~ 처음 부터... '},
  {'title': '<b>파이썬</b>인강 추천해주세요.',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=376128989&q

In [31]:
one_result['items'][0]['description']

"string = &quot;홀짝홀짝홀짝&quot;print(string[::2]) <b>파이썬</b>에서 이 코드를 실행시키면 '홀홀홀'이 나오는데 이유가 뭔가요?... 이 의미는 첫 번째 요소(<b>파이썬</b>의 인덱스는 0 부터 시작합니다) 부터 끝까지(-1 은 끝에서 첫 번째를 의미합니다.) 1... "

In [32]:
def delete_tag(input_str):
    input_str = input_str.replace("<b>", "")
    input_str = input_str.replace("</b>", "")
    
    return input_str

def get_description(pages):
    contents = []
    for sentences in pages['items']:
        contents.append(delete_tag(sentences['description']))
    
    return contents

In [33]:
contents = get_description(one_result)
contents

["string = &quot;홀짝홀짝홀짝&quot;print(string[::2]) 파이썬에서 이 코드를 실행시키면 '홀홀홀'이 나오는데 이유가 뭔가요?... 이 의미는 첫 번째 요소(파이썬의 인덱스는 0 부터 시작합니다) 부터 끝까지(-1 은 끝에서 첫 번째를 의미합니다.) 1... ",
 '파이썬으로 자격증따고싶어요!! 아직나이ㅣ어리니까 시간도많고 아이구조아라 파이썬시작하려면 뭐부터해야하나요?! 파이썬이 조금 익숙해지면 인터넷강의도... C언어 관계없이 파이썬 설치부터, 변수, 자료형 ~ 처음 부터... ',
 '파이썬인강 보면서 방학때 공부하려 하는데요 유튭은 보다가 답답해서 포기했어요 ㅜ 괜찮은 파이썬인강 사이트 아는것 있으면 추천좀 해주세요!!! 안녕하세요~ 파이썬인강을... 아무래도 파이썬을 공부하고 싶으신데 처음 접한다면... ',
 '파이썬을 처음 시작하게되었는데요. 게임코딩을 유튜브를 보면서 따라하고 있습니다. 파이썬을 이클립스로... 실행되지않고, 파이썬 자체에서도 실행되지 않습니다.... 무슨문제가 있고 왜 그런지 알려주세요ㅠㅠ  파이썬... ',
 '파이썬 과제좀 알려주세요ㅠㅠ  안녕하세요. 지식인에서도 과제풀이를 해주시는 분이 계시긴 하지만... C언어, 파이썬, 자바 등의 코딩이나 프로그래밍 작업에 대해 문의줘보실 수 있는데요. 관련하여 사이트... ',
 '... 이번에 파이썬 프로그래밍 언어에 도전을 해보려고 합니다 스크래치 코딩은 많이 해 보았는데 파이썬 같은 영어 타이핑 코딩은 처음이고 파이썬에 대해 아무것도 모르는 상태에서 시작을 했습니다 물론 파이썬보다 좋은 언어가... ',
 '1월에 파이썬 인강 들어보려 하는데 어떤 파이썬 인강 들어볼까요? 1. 비대면 강의: 비대면으로 실습할 수 있는 비대면라이브 강의가 있어요. 오프라인 학원 수업을 그대로 온라인으로 옮겨 놨다고 생각하면 될 것 같아요. 자세한 건 아래... ',
 '주말에 시간이 남아서 개인발전을 할려고 하다가 파이썬이 요즘 많이 사용... 지거든요 파이썬비대면수업

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t = Okt()
vectorizer = CountVectorizer(min_df=1)

In [35]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['string',
  '=',
  '&',
  'quot',
  ';',
  '홀짝',
  '홀짝',
  '홀짝',
  '&',
  'quot',
  ';',
  'print',
  '(',
  'string',
  '[::',
  '2',
  '])',
  '파이썬',
  '에서',
  '이',
  '코드',
  '를',
  '실행',
  '시키면',
  "'",
  '홀홀',
  '홀',
  "'",
  '이',
  '나오는데',
  '이유',
  '가',
  '뭔가',
  '요',
  '?...',
  '이',
  '의미',
  '는',
  '첫',
  '번째',
  '요소',
  '(',
  '파이썬',
  '의',
  '인덱스',
  '는',
  '0',
  '부터',
  '시작',
  '합니다',
  ')',
  '부터',
  '끝',
  '까지',
  '(-',
  '1',
  '은',
  '끝',
  '에서',
  '첫',
  '번',
  '째',
  '를',
  '의미',
  '합니다',
  '.)',
  '1',
  '...'],
 ['파이썬',
  '으로',
  '자격증',
  '따고싶어요',
  '!!',
  '아직',
  '나이',
  'ㅣ',
  '어리니까',
  '시간',
  '도',
  '많고',
  '아이구',
  '조아라',
  '파이썬',
  '시작',
  '하려면',
  '뭐',
  '부터',
  '해야하나요',
  '?!',
  '파이썬',
  '이',
  '조금',
  '익숙해지면',
  '인터넷',
  '강의',
  '도',
  '...',
  'C',
  '언어',
  '관계없이',
  '파이썬',
  '설치',
  '부터',
  ',',
  '변수',
  ',',
  '자료형',
  '~',
  '처음',
  '부터',
  '...'],
 ['파이썬',
  '인강',
  '보면서',
  '방학',
  '때',
  '공부',
  '하려',
  '하는데요',
  '유튭',
  '은',
  '보다가',
  '답답해서'

In [36]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize.append(sentence)

contents_for_vectorize

[" string = & quot ; 홀짝 홀짝 홀짝 & quot ; print ( string [:: 2 ]) 파이썬 에서 이 코드 를 실행 시키면 ' 홀홀 홀 ' 이 나오는데 이유 가 뭔가 요 ?... 이 의미 는 첫 번째 요소 ( 파이썬 의 인덱스 는 0 부터 시작 합니다 ) 부터 끝 까지 (- 1 은 끝 에서 첫 번 째 를 의미 합니다 .) 1 ...",
 ' 파이썬 으로 자격증 따고싶어요 !! 아직 나이 ㅣ 어리니까 시간 도 많고 아이구 조아라 파이썬 시작 하려면 뭐 부터 해야하나요 ?! 파이썬 이 조금 익숙해지면 인터넷 강의 도 ... C 언어 관계없이 파이썬 설치 부터 , 변수 , 자료형 ~ 처음 부터 ...',
 ' 파이썬 인강 보면서 방학 때 공부 하려 하는데요 유튭 은 보다가 답답해서 포기 했어요 ㅜ 괜찮은 파이썬 인강 사이트 아는것 있으면 추천 좀 해주세요 !!! 안녕하세요 ~ 파이썬 인강 을 ... 아무래도 파이썬 을 공부 하고 싶으신 데 처음 접 한 다 면 ...',
 ' 파이썬 을 처음 시작 하게 되었는데요 . 게임 코딩 을 유튜브 를 보면서 따라하고 있습니다 . 파이썬 을 이클립스 로 ... 실행 되지 않고 , 파이썬 자체 에서도 실행 되지 않습니다 .... 무슨 문제 가 있고 왜 그런지 알려주세요 ㅠㅠ 파이썬 ...',
 ' 파이썬 과제 좀 알려주세요 ㅠㅠ 안녕하세요 . 지식인 에서도 과제 풀이 를 해주시 는 분 이 계시긴 하지만 ... C 언어 , 파이썬 , 자바 등 의 코딩 이나 프로그래밍 작업 에 대해 문의 줘 보실 수 있는데요 . 관련 하여 사이트 ...',
 ' ... 이번 에 파이썬 프로그래밍 언어 에 도전 을 해보려고 합니다 스크래치 코딩 은 많이 해 보았는데 파이썬 같은 영어 타이핑 코딩 은 처음 이고 파이썬 에 대해 아무 것 도 모르는 상태 에서 시작 을 했습니다 물론 파이썬 보다 좋은 언어 가 ...',
 ' 1월 에 파이썬 인강 들어 보려 하는데 어떤 파이썬 인강 들어 볼까 요 ? 1 . 비대 면 강의 : 비 대

In [37]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<10x192 sparse matrix of type '<class 'numpy.int64'>'
	with 247 stored elements in Compressed Sparse Row format>

In [38]:
X.shape

(10, 192)

In [39]:
vectorizer.get_feature_names()

['1월',
 'arduino',
 'ccna',
 'import',
 'it',
 'led',
 'print',
 'quot',
 'read',
 'serial',
 'string',
 'ㅠㅠ',
 '강의',
 '같아요',
 '같은',
 '같이',
 '개인',
 '게임',
 '계시긴',
 '공부',
 '과제',
 '관계없이',
 '관련',
 '괜찮은',
 '교육',
 '국비',
 '그대로',
 '그런지',
 '깊이',
 '까지',
 '나오는데',
 '나이',
 '남아',
 '넓구요',
 '노란',
 '놨다',
 '늘면서',
 '답답해서',
 '대면',
 '대한',
 '대해',
 '도전',
 '되었는데요',
 '되지',
 '들어',
 '따고싶어요',
 '따라하고',
 '때문',
 '라이브',
 '리눅스',
 '만약',
 '많고',
 '많이',
 '모르는',
 '무슨',
 '문의',
 '문제',
 '물론',
 '뭔가',
 '받아서',
 '발전',
 '방학',
 '배우려는',
 '번째',
 '변수',
 '보다',
 '보다가',
 '보려',
 '보면서',
 '보실',
 '보았는데',
 '볼까',
 '부분',
 '부족합니다',
 '부터',
 '분야',
 '비대',
 '빅데이터',
 '빨간',
 '사용',
 '사이트',
 '상태',
 '생각',
 '설치',
 '수업',
 '수요',
 '스크래치',
 '시간',
 '시리얼',
 '시작',
 '시키면',
 '실습',
 '실행',
 '싶었는데',
 '싶으면',
 '싶으신',
 '아는것',
 '아래',
 '아무',
 '아무래도',
 '아이구',
 '아직',
 '안녕하세요',
 '않고',
 '않습니다',
 '알려주세요',
 '어떤',
 '어리니까',
 '언어',
 '에서',
 '에서도',
 '영어',
 '오프라인',
 '온라인',
 '옮겨',
 '요소',
 '요즘',
 '유튜브',
 '유튭',
 '으로',
 '의미',
 '이고',
 '이나',
 '이렇게',
 '이번',
 '이유',
 '이클립스',
 '익숙해지면',
 '인강',


In [40]:
new_post = ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 파이썬 을 배우는데 좋은 방법 이 어떤 것 인지 추천 해주세요']

In [41]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [42]:
# 유클리드 거리
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [43]:
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[7.14142842854285,
 6.708203932499369,
 6.48074069840786,
 6.48074069840786,
 5.5677643628300215,
 6.48074069840786,
 6.708203932499369,
 6.0,
 6.557438524302,
 6.6332495807108]

In [44]:
print("Best post is ", dist.index(min(dist)), "dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is  4 dist =  5.5677643628300215
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  파이썬 과제좀 알려주세요ㅠㅠ  안녕하세요. 지식인에서도 과제풀이를 해주시는 분이 계시긴 하지만... C언어, 파이썬, 자바 등의 코딩이나 프로그래밍 작업에 대해 문의줘보실 수 있는데요. 관련하여 사이트... 


In [45]:
# 거리를 구하는 또다른 방법 normailize

def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [46]:
dist = [dist_norm(each, new_post_vec) for each in X]

print("Best post is ", dist.index(min(dist)), "dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is  2 dist =  1.1105620748071996
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  파이썬인강 보면서 방학때 공부하려 하는데요 유튭은 보다가 답답해서 포기했어요 ㅜ 괜찮은 파이썬인강 사이트 아는것 있으면 추천좀 해주세요!!! 안녕하세요~ 파이썬인강을... 아무래도 파이썬을 공부하고 싶으신데 처음 접한다면... 


### TF IDF

In [47]:
# Tfidif
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf.sp.log(float(len(D))/len([doc for doc in D if t in doc]))
    return tf * idf

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [49]:
X = vectorizer.fit_transform(contents_for_vectorize)
X.shape

(10, 192)

In [50]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [51]:
dist = [dist_norm(each, new_post_vec) for each in X]

print("Best post is ", dist.index(min(dist)), "dist = ", min(dist))
print("test post is --> ", new_post)
print("best dist post is --> ", contents[dist.index(min(dist))])

Best post is  2 dist =  1.2547305646641695
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  파이썬인강 보면서 방학때 공부하려 하는데요 유튭은 보다가 답답해서 포기했어요 ㅜ 괜찮은 파이썬인강 사이트 아는것 있으면 추천좀 해주세요!!! 안녕하세요~ 파이썬인강을... 아무래도 파이썬을 공부하고 싶으신데 처음 접한다면... 
